# Financial Date Scraping
**_TABLE OF CONTENT_**
1. Installs, imports and Inputs  
    - Import 
    - Input 
2. Financial Data extraction  
    - Connect and write to database
    - Scrape Financials
3. Macro-Economic indicators extraction
    - Crude OIL, Treasury and Internation indices
    - Libor Data 
    - Write Fundamentals 

***
## 1. imports and Inputs

### 1.1 Import

In [1]:
from urllib.error import HTTPError
import numpy as np
import datetime
import pandas as pd
import yfinance as yf
from datetime import date, timedelta

import psycopg2
import requests
from bs4 import BeautifulSoup

print('Imported')

Imported


### 1.3 Inputs
- Read all ticker symbols:
    - List of the 20 companies with the highest market capitalization in the NYSE
- Read in list of all wanted fundamental indicators

>Date of composition 25/09/2020


In [2]:
#Create connection with database
def get_connection():
    connection = psycopg2.connect(user = "postgres",
                                  password = "postgres",
                                  host = "localhost",
                                  port = "5432",
                                  database = "postgres")
    return connection

#Close connection with database
def close_connection(connection):
    if connection:
        connection.close()
        print("Postgres connection is closed")

# Retrieves tickers from database, returns an array of strings
def get_tickers():
    tickers = []
    try:
        connection = get_connection()
        cursor = connection.cursor()
        select_query = """select ticker from companies"""
        cursor.execute(select_query,)
        records = cursor.fetchall()
        for row in records:
            tickers.append(row[0])
        close_connection(connection)
    except (Exception, psycopg2.Error) as error:
        print("Error while getting data", error)
    return tickers

In [3]:
#Initial reading to insert into database
#Writes to companies db and include: company name, symbol, size and sector
def write_companies():
    df = pd.read_csv("Companies.csv")
    try:
       connection = get_connection()
       cursor = connection.cursor()

       postgres_insert_query = """ INSERT INTO companies (ticker, name , sector, size) VALUES (%s,%s,%s,%s)"""
       i = 0
       for i in range(len(df.Name)):
           record_to_insert = (df['Ticker'][i], df['Name'][i], df['Sector'][i], df['Size'][i])
           cursor.execute(postgres_insert_query, record_to_insert)

           connection.commit()
           count = cursor.rowcount
           print (count, "Record inserted successfully into Companies table")

    except (Exception, psycopg2.Error) as error :
        if(connection):
            print("Failed to insert record into Companies table", error)

    finally:
        # Closing database connection.
        close_connection(connection)


***
## 2. Financial Data extraction
### 2.1 Connect and write to database

In [4]:
# Writes new financial data to database
def write_financials(df):
    try:
        connection = get_connection()
        cursor = connection.cursor()
        postgres_insert_query = """ INSERT INTO financials (ticker, date , open, close, volume , adjclose, high , low) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"""
        record_to_insert = (df['Ticker'][0],df["Date"][0], float(df['Open'][0]),float( df['Close'][0]), int(df['Volume'][0]),float(df['Adj Close'][0]), float(df['High'][0]), float(df['Low'][0]))
        cursor.execute(postgres_insert_query,record_to_insert)
        connection.commit()
        close_connection(connection)
    except (Exception, psycopg2.Error) as error:
        print("Error while getting data ", error)

### 2.2 Scrape financials

In [5]:
#Run Daily to insert into database, run below
def financials_main():
    for ticker in get_tickers():
         #Use yfinance to download financials
        df = yf.download(ticker, date.today())
        # Add scraped ticker symbol to financials
        df['Ticker'] = ticker
        # Reset index, date index becomes normal column
        df = df.reset_index()
        # Write to database
        write_financials(df)

***
## 3. Macro-Economic indicators extraction

### 3.1 Crude OIL, Treasury and Internation indices
 From Yahoo finance the following can be scraped
   - Crude oil CL=F
   - Brent Crude Oil: BZ= F
   - 13 week US Treasury Bill: ^IRX
   - Treasury Yield 5 Years: ^FVX
   - Treasury Yield 10 Years: ^TNX
   - Treasury Yield 30 Years: ^TYX
   - USD/EUR: EUR= X
   - USD/GBP: GBP= X
   - USD/MXN: MXN= X
   - USD/CNY: CNY= X
   - USD/JPY: JPY= X
   - USD/CAD: CAD= X
   - Gold
   - Bitcoin
   - DAX
   - Dow Jones Index
   - Nikkei Index

In [6]:
def write_macro_economics(df):
    yesterday = datetime.datetime.today() + timedelta(-1)
    try:
        #Create connection with database
        connection = get_connection()
        cursor = connection.cursor()
        postgres_insert_query = """ INSERT INTO macro_economic (date, crude_oil, brent_crude_oil,
        thirtheen_week_treasury_bill, treasury_yield_five, treasury_yield_ten,
        treasury_yield_thirty, dji, dax, nikkei, gold, bitcoin, usd_eur, usd_gbp, usd_cad,
        usd_jpy, usd_cny, usd_aud, usd_mxn)
        VALUES (%s,%s,%s,%s,%s,%s,%s, %s,%s,%s,%s,%s,%s,%s, %s,%s,%s,%s,%s)"""
        record_to_insert = (yesterday, df['CrudeOil'][0],df["BrentOil"][0],
                            df['ThirtheenWeekTB'][0], df['TreasuryYieldFiveYears'][0],
                            df['TreasuryYieldTenYears'][0],df['TreasuryYieldThirtyYears'][0],
                            df['DJI'][0], df['DAX'][0], df['NI225'][0], df['Gold'][0],
                            df['Bitcoin'][0], df['USD/EUR'][0], df['USD/GBP'][0], df['USD/CAD'][0],
                            df['USD/JPY'][0], df['USD/CNY'][0], df['USD/AUD'][0], df['USD/MXN'][0])
        cursor.execute(postgres_insert_query,record_to_insert)
        connection.commit()
        close_connection(connection)
    except (Exception, psycopg2.Error) as error:
        print("Error while getting data ", error)



In [7]:
def macro_economic_main():
    df = yf.download('CL=F', date.today()).rename(columns = {'Close':'CrudeOil'})
    macro = pd.DataFrame(df['CrudeOil'])
    macro['BrentOil'] = pd.DataFrame(yf.download('BZ=F', date.today())).Close
    macro['ThirtheenWeekTB'] = pd.DataFrame(yf.download('^IRX', date.today())).Close
    macro['TreasuryYieldFiveYears'] = pd.DataFrame(yf.download('^FVX', date.today())).Close
    macro['TreasuryYieldTenYears']  = pd.DataFrame(yf.download('^TNX', date.today())).Close
    macro['TreasuryYieldThirtyYears'] = pd.DataFrame(yf.download('^TYX', date.today())).Close
#     macro['ThirtheenWeekTB'] = np.NaN
#     macro['TreasuryYieldFiveYears'] = np.NaN
#     macro['TreasuryYieldTenYears']  = np.NaN
#     macro['TreasuryYieldThirtyYears'] = np.NaN
    macro['DJI'] = pd.DataFrame(yf.download('^DJI', date.today())).Close[0]
    macro['DAX'] = pd.DataFrame(yf.download('^DAX-EU', date.today())).Close[0]
    macro['NI225'] = pd.DataFrame(yf.download('^N225', date.today())).Close[0]
    macro['Gold'] = pd.DataFrame(yf.download('GC=F', date.today())).Close[0]
    macro['Bitcoin'] = pd.DataFrame(yf.download('BTC-USD', date.today())).Close[0]
    macro['USD/EUR'] = pd.DataFrame(yf.download('EUR=X', date.today())).Close[0]
    macro['USD/GBP'] = pd.DataFrame(yf.download('GBP=X', date.today())).Close[0]
    macro['USD/CAD'] = pd.DataFrame(yf.download('CAD=X', date.today())).Close[0]
    macro['USD/JPY'] = pd.DataFrame(yf.download('JPY=X', date.today())).Close[0]
    macro['USD/CNY'] = pd.DataFrame(yf.download('CNY=X', date.today())).Close[0]
    macro['USD/AUD'] = pd.DataFrame(yf.download('AUD=X', date.today())).Close[0]
    macro['USD/MXN'] = pd.DataFrame(yf.download('MXN=X', date.today())).Close[0]
    write_macro_economics(macro)

### 3.2 Libor data

In [8]:
#Update certain macro-economic row with Overnight Libor rate
def write_libor(df):
    try:
        #Create connection with database
        connection = get_connection()
        cursor = connection.cursor()
        update_query = """update macro_economic set libor = %s where date = %s"""
        value = float(str(df['Euro LIBOR - overnight'][1]).rstrip('%'))
        cursor.execute(update_query, (value, df[df.columns[-1]][1]))
        update_gdp = """update macro_economic set gdp = %s where date = %s"""
        cursor.execute(update_gdp, (-0.314, df[df.columns[-1]][1]))
        connection.commit()
        close_connection(connection)
    except (Exception, psycopg2.Error) as error:
        print("Error while getting data ", error)

In [9]:
# Scrapes global-rates.com to extract libor rates
def scrape_libor_main():
    url = 'https://www.global-rates.com/en/interest-rates/libor/libor.aspx'
    r = requests.get(url)
    html = r.text
    soup = BeautifulSoup(html)
    table = soup.find('table', {"style": "width:100%;margin:10px 0px 0px 0px;border:1px solid #CCCCCC;"})
    rows = table.find_all('tr')
    data = []
    for row in rows[1:]:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele])

    result = pd.DataFrame(data)
    result = result.transpose()
    result.head()
    result = result.drop([2,6,7,9,10,11,12,13], axis = 1)
    result['date'] = date.today() + timedelta(-1)
    result['date'][2] = date.today() + timedelta(-2)
    result['date'][3] = date.today() + timedelta(-3)
    result['date'][4] = date.today() + timedelta(-4)
    result['date'][5] = date.today() + timedelta(-5)
    #grab the first row for the header
    new_header = result.iloc[0]
    #take the data less the header row
    result = result[1:]
    result.columns = new_header
    write_libor(result)



Write Fundamentals

In [10]:
def write_fundamentals(ticker ,d):
    yesterday = datetime.datetime.today() + timedelta(-1)
    try:
        connection = get_connection()
        cursor = connection.cursor()
        postgres_insert_query = """ INSERT INTO fundamentals (ticker, date , previousclose, regularmarketopen,
        twoHundredDayAverage , trailingAnnualDividendYield, payoutRatio , regularMarketDayHigh, averageDailyVolume10Day, regularMarketPreviousClose,
        fiftyDayAverage,trailing_Annual_Dividend_Rate, averageVolume10days, dividendRate , beta,
        regularMarketDayLow, trailing_PE, regularMarketVolume, marketCap, averageVolume,
        price_to_sales_12months, dayLow, ytdReturn, askSize, fiftyTwo_Week_High, forwardPE,
        five_years_average_dividend_yield, ask, bid,  fiftyTwoWeekLow, dividendYield, bidSize, dayHigh,
        entreprise_to_revenue, beta3Year, profitMargins, entreprise_to_ebitda , fiftytwoweekchange,
        forwardEps, shares_outstanding, bookValue, shares_short,
        shares_Percent_Shares_Out, last_Fiscal_Year_End, held_Percent_Institutions, net_Income_To_Common, trailing_Eps, SandP_fiftytwo_Week_Change,
        price_To_Book, held_Percent_Insiders, next_Fiscal_Year_End, most_Recent_Quarter, short_Ratio, shares_Short_Previous_Month_Date,
        float_Shares, enterprise_Value, three_Year_Average_Return, date_Short_Interest,
       peg_Ratio, short_Percent_Of_Float, shares_Short_Prior_Month, five_Year_Average_Return, regular_Market_Price) VALUES (%s,%s,%s,%s,%s,%s,%s,%s, %s,%s,%s,%s,%s,%s,%s, %s,%s,%s,%s,%s,%s,%s,%s, %s,%s,%s,%s,%s,%s,%s,%s, %s,%s,%s,%s,%s,%s,%s,%s, %s,%s,%s,%s,%s,%s,%s,%s, %s,%s,%s,%s,%s,%s,%s,%s, %s,%s,%s,%s,%s,%s,%s,%s )"""
        record_to_insert = (ticker ,yesterday, d.get("previousClose") ,d.get("regularMarketOpen"), d.get("twoHundredDayAverage"),
        d.get("trailingAnnualDividendYield"), d.get("payoutRatio"), d.get("regularMarketDayHigh"), d.get('averageDailyVolume10Day'),d.get('regularMarketPreviousClose'), d.get('fiftyDayAverage'),
        d.get('trailingAnnualDividendRate'), d.get('averageVolume10days'),d.get( 'dividendRate')  , d.get('beta'), d.get('regularMarketDayLow'), d.get('trailingPE'), d.get('regularMarketVolume'), d.get('marketCap'), d.get('averageVolume'),
        d.get('priceToSalesTrailing12Months'), d.get('dayLow'), d.get('ytdReturn'), d.get('askSize'), d.get('fiftyTwoWeekHigh'), d.get('forwardPE'),
        d.get('fiveYearAvgDividendYield'), d.get('ask'), d.get('bid'),  d.get('fiftyTwoWeekLow'), d.get('dividendYield'), d.get('bidSize'), d.get('dayHigh'),
        d.get('enterpriseToRevenue'), d.get('beta3Year'), d.get('profitMargins'), d.get('enterpriseToEbitda') , d.get('52WeekChange'),
        d.get('forwardEps'), d.get('sharesOutstanding'), d.get('bookValue'), d.get('sharesShort'), d.get('sharesPercentSharesOut'), d.get('lastFiscalYearEnd') , d.get('heldPercentInstitutions'), d.get('netIncomeToCommon'), d.get('trailingEps'), d.get('SandP52WeekChange'),
        d.get('priceToBook'), d.get('heldPercentInsiders'), d.get('nextFiscalYearEnd'), d.get('mostRecentQuarter'), d.get('shortRatio'), d.get('sharesShortPreviousMonthDate'),
        d.get('floatShares'), d.get('enterpriseValue'), d.get('threeYearAverageReturn'), d.get('dateShortInterest'), d.get('pegRatio'), d.get('shortPercentOfFloat'), d.get('sharesShortPriorMonth'), d.get('fiveYearAverageReturn'), d.get('regularMarketPrice'))
        cursor.execute(postgres_insert_query,record_to_insert)
        connection.commit()
        close_connection(connection)
    except (Exception, psycopg2.Error) as error:
        print("Error while getting data ", error)

In [11]:
def scrape_fundamentals_main():
    for t in get_tickers():
        try:
            tck = yf.Ticker(t)
            d = tck.get_info()
            write_fundamentals(t, d)
        except IndexError :
            print('Index error for {}'.format(t))

## Run daily

In [12]:
print('Downloading Financials')
#Write all financials to financials table
financials_main() #Prints out failed tickers

print('\n Downloading Fundamentals ')
#Write all fundamentals to fundamentals table
scrape_fundamentals_main()
#Print out failed tickers

print('\n Download Macro-Economic Data ')
#Update created macro-economic table
macro_economic_main()
#Update created macro-economic table
scrape_libor_main()

print('\n Done for today ({})'.format(datetime.datetime.today()))

Postgres connection is closed
[*********************100%***********************]  1 of 1 completed
Postgres connection is closed
[*********************100%***********************]  1 of 1 completed
Postgres connection is closed
[*********************100%***********************]  1 of 1 completed
Postgres connection is closed
[*********************100%***********************]  1 of 1 completed
Postgres connection is closed
[*********************100%***********************]  1 of 1 completed
Postgres connection is closed
[*********************100%***********************]  1 of 1 completed
Postgres connection is closed
[*********************100%***********************]  1 of 1 completed
Postgres connection is closed
[*********************100%***********************]  1 of 1 completed
Postgres connection is closed
[*********************100%***********************]  1 of 1 completed
Postgres connection is closed
[*********************100%***********************]  1 of 1 completed
Postgres c